# Prep

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    from google.colab import userdata
    ghub_key = userdata.get('ghub')
    dbhub_key = userdata.get('dbhub')
    import os,sys
    from google.colab import auth
    auth.authenticate_user()
    
    import gspread
    from google.auth import default
    creds, _ = default()
    
    gc = gspread.authorize(creds)
except:pass

In [ ]:
import os,sys,json,pwd
from copy import deepcopy as dc

In [ ]:
os.system(f"{sys.executable} -m pip install --upgrade mystring[all] pandas hugg[all] tqdm ephfile matplotlib pyplotlib numpy matplotlib venny4py plotly pnostic kaleido")

import plotly.graph_objects as go
import mystring,pandas as pd,hugg
from ephfile import ephfile
import matplotlib.pyplot as plt
import numpy as np
import mystring as mys
from ephfile import ephfile
import matplotlib.pyplot as plt
from venny4py.venny4py import *
from tqdm import tqdm
%matplotlib inline

In [ ]:
def flatten_column_header(column_header):
    if isinstance(column_header, list) or isinstance(column_header, tuple):
        return ' '.join([str(ch) for ch in column_header]).strip()
    return column_header

In [ ]:
#https://gist.github.com/ines/04b47597eb9d011ade5e77a068389521
def diff_strings(a, b):
    import difflib
    try:
        from wasabi import color
    except:
        os.system("{0} -m pip install wasabi".format(sys.executable))
        from wasabi import color
    output = []
    matcher = difflib.SequenceMatcher(None, a, b)
    for opcode, a0, a1, b0, b1 in matcher.get_opcodes():
        if opcode == "equal":
            output.append(a[a0:a1])
        elif opcode == "insert":
            output.append(color(b[b0:b1], fg=16, bg="green"))
        elif opcode == "delete":
            output.append(color(a[a0:a1], fg=16, bg="red"))
        elif opcode == "replace":
            output.append(color(b[b0:b1], fg=16, bg="green"))
            output.append(color(a[a0:a1], fg=16, bg="red"))
    return "".join(output)


In [ ]:
class redirect(object):
    def __init__(self):
        self.stream = None
        self.f = None
    def __enter__(self):
        self.stream = sys.stdout
        self.f = open(os.devnull, 'w')
        sys.stdout = self.f
        return self
    def __exit__(self,a=None,b=None,c=None):
        sys.stdout = self.stream


In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks"
path_two = "/drive/MyDrive/Colab Notebooks"

if os.path.exists(path) or os.path.exists(path_two):
    if os.path.exists(path):
        os.chdir(path)
    else:
        os.chdir(path)

    class mitosheet(object):
        def __init__(self):
            pass
        def sheet(self, *args, **kwargs):
            print("Not available")
else:
    try:
        import mitosheet
    except:
        os.system(f"{sys.executable} -m pip install mitosheet && {sys.executable} -m mitosheet install")
        import mitosheet

In [ ]:
# Load the PyCryptoBench File

class py_util(object):
    def __init__(self):
        self.foil = "PyCryptoBench.py"
    def __enter__(self):
        import hugg;
        return hugg.ghub("franceme/pnostic_rsc", ghub_key).impor("providers/"+self.foil, delete=True).app
    def __exit__(self, a=None,b=None,c=None):
        if os.path.exists(self.foil):
            os.remove(self.foil)

In [ ]:
# Load the PyCryptoBench File

#with hugg.eph_mgr(hugg.ghub('franceme/pnostic_rsc', 'X'), 'runners/llm/__init__.py') as mgmt:
#	fieldz = {x:'Field_'+str(x) for x in mgmt.files['__init__'].total_fields}


In [ ]:
# Get Current User

def get_username():
	return str(pwd.getpwuid(os.getuid())[0])

In [ ]:
#https://sparkbyexamples.com/pandas/pandas-add-column-based-on-another-column/

def percents(db):
    for value in ["Precision", "Recall", "Accuracy", "FOne"]:
        if value in db.columns:
            db[value] = pd.Series(["{0:.2f}%".format(val * 100) for val in db[value]], index = db.index)
    return db

def rename_this(db):
    rename_mapping = [
        ("Tp sum", "TP"),
        ("Fp sum", "FP"),
        ("Tn sum", "TN"),
        ("Fn sum", "FN"),
        ("FOne", "F1"),
    ]

    for kol in db.columns:
        if kol.startswith("BenchmarkRule count"):
            rename_mapping += [
                (kol, kol.replace("BenchmarkRule count", "Rule").strip().replace(".0","").strip())
            ]
        else:
            rename_mapping += [
                (kol, kol.replace("count","").title().strip())
            ]

    for (rename_from, rename_to) in rename_mapping:
        if rename_from in db.columns:
            db = db.rename(columns={rename_from: rename_to})
    return db

def precision(db, tp="Tp sum", fp="Fp sum", tn="Tn sum", fn="Fn sum"):
    #db["Precision"] = db[tp]/(db[tp]+db[fp])
    db = db.assign(Precision=lambda x:x[tp]/(x[tp]+x[fp]))
    return db

def recall(db, tp="Tp sum", fp="Fp sum", tn="Tn sum", fn="Fn sum"):
    #db["Recall"] = db[tp]/(db[tp]+db[fn])
    db = db.assign(Recall=lambda x:x[tp]/(x[tp]+x[fn]))
    return db

def accuracy(db, tp="Tp sum", fp="Fp sum", tn="Tn sum", fn="Fn sum"):
    #db["Accuracy"] = (db[tp]+db[tn])/(db[tp]+db[fp]+db[tn]+db[fn])
    db = db.assign(Accuracy=lambda x:(x[tp]+x[tn])/(x[tp]+x[fp]+x[tn]+x[fn]))
    return db

def f_one(db, tp="Tp sum", fp="Fp sum", tn="Tn sum", fn="Fn sum"):
    db = precision(db)
    db = recall(db)
    #db["F1"] = 2*((db["Precision"]*db["Recall"])/(db["Precision"]+db["Recall"]))
    db = db.assign(FOne=lambda x:2*((x["Precision"]*x["Recall"])/(x["Precision"]+x["Recall"])))
    return db

def sums(db, tp="Tp sum", fp="Fp sum", tn="Tn sum", fn="Fn sum"):
    #db["Sum"] = db[tp]+db[fp]+db[tn]+db[fn]
    db = db.assign(Sum=lambda x:x[tp]+x[fp]+x[tn]+x[fn])
    return db

def just_pretty(db):
    return rename_this(percents(db))

def set_all(db, tp="Tp sum", fp="Fp sum", tn="Tn sum", fn="Fn sum", use_percents=False, rename=False, pretty=False):
    db = accuracy(db,tp,fp,tn,fn)
    db = f_one(db,tp,fp,tn,fn)
    db = sums(db,tp,fp,tn,fn)
    if use_percents or pretty:
        db = percents(db)
    if rename or pretty:
        db = rename_this(db)
    return db

In [ ]:
def re_order(df1_pivot):
    # Reordered column Tp sum
    if 'Tp sum' in df1_pivot.columns:
        df1_pivot_columns = [col for col in df1_pivot.columns if col != 'Tp sum']
        df1_pivot_columns.insert(2, 'Tp sum')
        df1_pivot = df1_pivot[df1_pivot_columns]

    if 'Fp sum' in df1_pivot.columns:
        # Reordered column Fp sum
        df1_pivot_columns = [col for col in df1_pivot.columns if col != 'Fp sum']
        df1_pivot_columns.insert(3, 'Fp sum')
        df1_pivot = df1_pivot[df1_pivot_columns]

    if 'Tn sum' in df1_pivot.columns:
        # Reordered column Tn sum
        df1_pivot_columns = [col for col in df1_pivot.columns if col != 'Tn sum']
        df1_pivot_columns.insert(4, 'Tn sum')
        df1_pivot = df1_pivot[df1_pivot_columns]

    if 'Fn sum' in df1_pivot.columns:
        # Reordered column Tn sum
        df1_pivot_columns = [col for col in df1_pivot.columns if col != 'Fn sum']
        df1_pivot_columns.insert(5, 'Fn sum')
        df1_pivot = df1_pivot[df1_pivot_columns]

    return df1_pivot

In [ ]:
def printz(db):
    print(db.to_csv())
    print("="*50)
    print(db.to_markdown())

In [ ]:
fc = lambda frame: mystring.frame(dc(frame))

# Prep Staging

## Imports

In [ ]:
from IPython.display import clear_output
import pandas as pd,datetime,json
import pickle
try:
    from ipywidgets import *
    import pyplotlib
except:
    os.system("{0} -m pip install --upgrade ipywidgets pyplotlib".format(sys.executable))
    from ipywidgets import *
    import pyplotlib

## Setup

In [ ]:
cleaned_values = {}

In [ ]:
def stash(dyct={}):
    folder = lambda x:"stashing/results_"+str(x)
    folder_number = 0

    while os.path.exists(folder(folder_number)):
        folder_number += 1

    fldr = folder(folder_number)
    os.system("mkdir -p {0}".format(fldr))

    with open(os.path.join(fldr, "__init__.py"), "w+") as core_py:
        core_py.write("""#!/usr/bin/env python3
import os,sys
local_path = lambda x:os.path.join(os.path.abspath(os.curdir), x)
stash_date = "{0}"
foils = [
""".format(mys.now_utc_to_iso()))
        for llm_type,llm_values in dyct.items():
            if isinstance(llm_values, dict):
                for key,data in llm_values.items():
                    data_name = f"{fldr}/{llm_type}_{key}"
                    if isinstance(data, pd.DataFrame):
                        data_name += ".csv"
                        data.to_csv(data_name)
                    else:
                        data_name += ".json"
                        with open(data_name, "w+") as writer:
                            json.dump(data, writer)
                    core_py.write("\tlocal_path('{0}'),\n".format(os.path.basename(data_name)))
            elif isinstance(llm_values, pd.DataFrame):
                data_name = f"{fldr}/{llm_type}_frame.csv"
                llm_values.to_csv(data_name)
                core_py.write("\tlocal_path('{0}'),\n".format(os.path.basename(data_name)))
        core_py.write("]")

In [ ]:
plt.style.use([
    pyplotlib.styles['ipynb'],
    #pyplotlib.styles['colorsblind34'],
    #pyplotlib.styles['science'],
    #pyplotlib.styles['ieee'],
])

#pyplotlib.reset()

# Compiled Sets

## Nim Experimental live set

In [ ]:
#Strings Examples: https://nim-by-example.github.io/strings/
#stdutils : https://nim-lang.org/docs/strutils.html
#https://docs.w3cub.com/nim/strutils

In [ ]:
#curl https://nim-lang.org/choosenim/init.sh -sSf | sh
if not os.path.exists("/{0}/.nimble/bin/".format(get_username())):
    os.system("curl https://nim-lang.org/choosenim/init.sh -sSf | bash -s -- -y")


In [ ]:
if not os.path.exists("/bin/nimble") and os.path.exists("/{0}/.nimble/bin/".format(get_username())):
    for x in os.listdir("/{0}/.nimble/bin/".format(get_username())):
        cmd = f"ln -s /{get_username()}/.nimble/bin/{os.path.basename(x)} /bin/{os.path.basename(x)}"
        print(cmd);os.system(cmd)


In [ ]:
#curl https://nim-lang.org/choosenim/init.sh -sSf | sh
#Soft link everything into /bin
try:
    import nimporter
except:
    os.system("{0} -m pip install --upgrade nimporter".format(sys.executable))
    import nimporter

In [ ]:
%%writefile sample.nim

import nimpy

proc add(a:int, b:int): int {.exportpy.} = 
    return a+b

In [ ]:
!nimporter compile

In [ ]:
import sample
sample.add(6,7)

## Rust Compiled Sample !! It'd take too long

# Rest